In [1]:
import pandas as pd
import numpy as np
import re
import mysql.connector as sql
import pymysql
from sqlalchemy import create_engine

In [2]:
file_dir = 'D:\Jobseeking\CAD'
#insert your mysql credential here
db_name = 'movies'
db_user = 'root'
db_pass = 'pass'
db_host = 'localhost'

#Read the file as Pandas DataFrames
df = pd.read_csv(f'{file_dir}/movies.csv', low_memory=False)
df = df.replace('\n','',regex=True)
#transform name of the columns to make it more readable
df.columns= df.columns.str.lower()
print(df.size)
df.head()

89991


,movies,year,genre,rating,one-line,stars,votes,runtime,gross
0,Blood Red Sky,(2021),"Action, Horror, Thriller",6.1,A woman with a mysterious illness is forced in...,Director:Peter Thorwarth| Stars:Peri B...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"Animation, Action, Adventure",5.0,The war for Eternia begins again in what may b...,"Stars:Chris Wood, Sarah Michel...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,"Stars:Andrew Lincoln, Norman R...","885,805",44.0,NaN
3,Rick and Morty,(2013– ),"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits o...,"Stars:Justin Roiland, Chris Pa...","414,849",23.0,NaN
4,Army of Thieves,(2021),"Action, Crime, Horror",NaN,"A prequel, set before the events of Army of th...",Director:Matthias Schweighöfer| Stars:...,NaN,NaN,NaN


In [3]:
#Filter out TV Show
df = df.drop(df[df[['movies','year']].duplicated()].index)
df = df.drop(df[df['year'].str.contains('–',na=False)].index).reset_index(drop=True)
df.head()

,movies,year,genre,rating,one-line,stars,votes,runtime,gross
0,Blood Red Sky,(2021),"Action, Horror, Thriller",6.1,A woman with a mysterious illness is forced in...,Director:Peter Thorwarth| Stars:Peri B...,"21,062",121.0,NaN
1,Army of Thieves,(2021),"Action, Crime, Horror",NaN,"A prequel, set before the events of Army of th...",Director:Matthias Schweighöfer| Stars:...,NaN,NaN,NaN
2,The Last Letter from Your Lover,(2021),"Drama, Romance",6.8,A pair of interwoven stories set in the past a...,Director:Augustine Frizzell| Stars:Sha...,"5,283",110.0,NaN
3,Gunpowder Milkshake,(2021),"Action, Adventure, Thriller",6.0,Three generations of women fight back against ...,Director:Navot Papushado| Stars:Karen ...,"17,989",114.0,NaN
4,Fear Street: 1994,(2021),"Drama, Horror, Mystery",6.2,A circle of teenage friends accidentally encou...,Director:Leigh Janiak| Stars:Kiana Mad...,"50,148",107.0,NaN


In [4]:
print(df.isna().sum())
print(df.dtypes)

movies         0
year         479
genre         71
rating       819
one-line       0
stars          0
votes        819
runtime      906
gross       4360
dtype: int64
movies       object
year         object
genre        object
rating      float64
one-line     object
stars        object
votes        object
runtime     float64
gross        object
dtype: object


In [5]:
#Clean the data
#Handle Missing Values
mis_col = [column for column in df.columns if df[column].isnull().sum() > 0]
mis_col.remove('genre')

#1 Clean the year column
df['year'] = df['year'].str.extract('(\d+)', expand=False).fillna(-1).astype(int)
df['year'] = df['year'].replace(-1, np.nan).astype('Int64')

#2 Clean the votes column
df['votes'] = df['votes'].str.extract('(\d+)', expand=False).fillna(-1).astype(int)
df['votes'] = df['votes'].replace(-1, np.nan).astype('Int64')

#3 Clean the gross column
df['gross'] = df['gross'].str.extract('(\d+)', expand=False).fillna(-1).astype(int)
df['gross'] = df['gross'].replace(-1, np.nan).astype(float)

#4 Clean the genre column
df['genre'] = df['genre'].str.strip()
df['genre'] = df['genre'].str.split(', ')

#5 Clean the star column
stars = df['stars']
stars = stars.str.split('|',expand=True)
stars.rename({0:'director',1:'cast'},axis='columns',inplace=True)
temp = stars[stars['director'].str.contains('Star',na=False)]['director'].copy()
no_direct = list(stars[stars['director'].str.contains('Star',na=False)].index) #film with no director info
stars.iloc[no_direct] = np.nan
stars['cast'].iloc[no_direct] = temp
#removing 'director' and 'stars'
stars['director'] = stars['director'].str.replace(r'    Director:|    Directors:','',regex=True)
stars['cast']     = stars['cast'].str.replace(r'     Star:|     Stars:','',regex=True)
stars['director'] = stars['director'].str.strip()
stars['cast']     = stars['cast'].str.strip()
df['director']    = stars['director'].str.split(', ')
df['cast']        = stars['cast'].str.split(', ')
df = df.drop('stars',axis='columns')
df.head()

,movies,year,genre,rating,one-line,votes,runtime,gross,director,cast
0,Blood Red Sky,2021,"[Action, Horror, Thriller]",6.1,A woman with a mysterious illness is forced in...,21,121.0,NaN,[Peter Thorwarth],"[Peri Baumeister, Carl Anton Koch, Alexander S..."
1,Army of Thieves,2021,"[Action, Crime, Horror]",NaN,"A prequel, set before the events of Army of th...",<NA>,NaN,NaN,[Matthias Schweighöfer],"[Matthias Schweighöfer, Nathalie Emmanuel, Rub..."
2,The Last Letter from Your Lover,2021,"[Drama, Romance]",6.8,A pair of interwoven stories set in the past a...,5,110.0,NaN,[Augustine Frizzell],"[Shailene Woodley, Joe Alwyn, Wendy Nottingham..."
3,Gunpowder Milkshake,2021,"[Action, Adventure, Thriller]",6.0,Three generations of women fight back against ...,17,114.0,NaN,[Navot Papushado],"[Karen Gillan, Lena Headey, Carla Gugino, Mich..."
4,Fear Street: 1994,2021,"[Drama, Horror, Mystery]",6.2,A circle of teenage friends accidentally encou...,50,107.0,NaN,[Leigh Janiak],"[Kiana Madeira, Olivia Scott Welch, Benjamin F..."


In [6]:
#2 clean the votes column
#df['genre'] = df['genre'].fillna(-1).astype(str)
#df['genre'] = df['genre'].replace('-1', np.nan).astype(str)

In [7]:
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(user= db_user,
                               host = db_host,
                               pw= db_pass,
                               db= db_name))


In [8]:
#MOVIE TABLE

movie_columns = ['movies','year','rating','one-line','votes','runtime','gross']
movie_df = df[movie_columns]
movie_df.rename({'movies':'title','one-line':'description','votes':'vote_count',
                            'runtime':'running_time_minutes', 'gross':'gross_millions'},
                            axis='columns', inplace=True)

#Insert Movie Table
movie_df.to_sql('movie', con = engine, if_exists = 'append', chunksize = 5000,index=False)
movie_df.head()

C:\Users\Windows X\.conda\envs\Project\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,title,year,rating,description,vote_count,running_time_minutes,gross_millions
0,Blood Red Sky,2021,6.1,A woman with a mysterious illness is forced in...,21,121.0,NaN
1,Army of Thieves,2021,NaN,"A prequel, set before the events of Army of th...",<NA>,NaN,NaN
2,The Last Letter from Your Lover,2021,6.8,A pair of interwoven stories set in the past a...,5,110.0,NaN
3,Gunpowder Milkshake,2021,6.0,Three generations of women fight back against ...,17,114.0,NaN
4,Fear Street: 1994,2021,6.2,A circle of teenage friends accidentally encou...,50,107.0,NaN


In [9]:
#PERSON TABLE
#Getting all different names
name = df['cast'].explode('cast').drop_duplicates()
name = name.append(df['director'].explode('director').drop_duplicates())
name = name.drop_duplicates().reset_index(drop=True)
name = pd.DataFrame({'name':list(name)}).dropna().drop_duplicates()

#Insert person table
name.to_sql('person', con = engine, if_exists = 'append', chunksize = 5000,index=False)
name.head()

,name
0,Peri Baumeister
1,Carl Anton Koch
2,Alexander Scheer
3,Kais Setti
4,Matthias Schweighöfer


In [10]:
#GENRE TABLE
#Getting all different genres
genre_name = df['genre'].explode('genre').drop_duplicates()
genre_name = genre_name.reset_index(drop=True)
genre_name = pd.DataFrame({'genre_name':list(genre_name)}).dropna().drop_duplicates()
genre_name.to_sql('genre', con = engine, if_exists = 'append', chunksize = 1000,index=False)
genre_name.head()

,genre_name
0,Action
1,Horror
2,Thriller
3,Crime
4,Drama


In [11]:
#Query for selecting movie table
db_con = sql.connect(host=db_host, database=db_name, user=db_user, password=db_pass)
try:
    SQL_Query = pd.read_sql_query(
        '''select
          *
          from movie''', db_con)

    mov_df = pd.DataFrame(SQL_Query, columns=['ID', 'title','year'])
    #print(mov_df)
except:
    print("Error: unable to convert the data")
db_con.close()

In [12]:
#MOVIE_GENRE TABLE
#Setting up the movie_genre(mg) dataframe/table
mg = df[['movies','year','genre']].explode('genre')

#Query for genre table
db_con = sql.connect(host=db_host, database=db_name, user=db_user, password=db_pass)
try:
    SQL_Query = pd.read_sql_query(
        '''select
          *
          from genre''', db_con)

    temp = pd.DataFrame(SQL_Query, columns=['ID', 'genre_name'])
    #print(temp)
except:
    print("Error: unable to convert the data")
db_con.close()
    
#Get movieID and genreID based on title,year, and genre_name
temp = temp.drop_duplicates(subset=['genre_name']).set_index('genre_name')
mg['genreID'] = mg.genre.map(temp['ID'].to_dict()).astype('Int64')
mov_ge_df = mov_df.drop_duplicates(subset=['title','year']).set_index(['title','year'])
mg['movieID'] = pd.Series(list(zip(mg.movies,mg.year))).map(mov_ge_df['ID'].to_dict())
mg = mg[['movieID','genreID']].drop_duplicates(subset=['movieID','genreID'])
mg = mg.drop(list(mg[mg.genreID.isna()].index),axis='rows')

#Insert movie genre table
mg.to_sql('movie_genre', con = engine, if_exists = 'append', chunksize = 5000,index=False)
mg.head()

,movieID,genreID
0,1,1
0,1,2
0,1,3
1,1,4
2,1,5


In [13]:
#MOVIE_PEOPLE TABLE
#Setting up the movie_people(mp) dataframe/table
mp = df[['movies','year','cast']].explode('cast').rename({'cast':'person'},axis='columns')
mp['profession'] = 'cast'
temp = df[['movies','year','director']].explode('director').rename({'director':'person'},axis='columns')
temp['profession'] = 'director'
mp = mp.append(temp).reset_index()
mp = mp.drop(list(mp[mp.person.isna()].index),axis='rows').reset_index(drop=True)
#mp.set_index(['movies','year'])

#Query for person table
db_con = sql.connect(host=db_host, database=db_name, user=db_user, password=db_pass)
try:
    SQL_Query = pd.read_sql_query(
        '''select
          *
          from person''', db_con)

    temp = pd.DataFrame(SQL_Query, columns=['ID', 'name'])
    #print(temp)
except:
    print("Error: unable to convert the data")
    

db_con.close()

#Get movieID and PersonID based on title,year, and person name
temp = temp.drop_duplicates(subset=['name']).set_index('name')
mp['personID'] = mp.person.map(temp['ID'].to_dict())
mov_df = mov_df.drop_duplicates(subset=['title','year']).set_index(['title','year'])
mp['movieID'] = pd.Series(list(zip(mp.movies,mp.year))).map(mov_df['ID'].to_dict())
mp = mp[['movieID','personID','profession']].drop_duplicates(subset=['movieID','personID','profession'])

#Insert movie person table
mp.to_sql('movie_people', con = engine, if_exists = 'append', chunksize = 5000,index=False)
mp.head()

,movieID,personID,profession
0,1,1,cast
1,1,2,cast
2,1,3,cast
3,1,4,cast
4,2,5,cast
